In [1]:
!pip install keras
!pip install tensorflow

# IMAGE PREPROCESSING

In [2]:
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='H118gyqPie94forcf8S88nwhWCnKYTbHrk5L_aTQaPxo',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'arrhythmiaclassification-donotdelete-pr-afnighsy3hw2ca'
object_key = 'Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
#unzipping your data file
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
pwd

'/home/wsuser/work'

In [7]:
import os
filenames=os.listdir('/home/wsuser/work/data/train')

In [8]:
 from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
# setting parameters for data augmentation for training data
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [10]:
# data augmentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

In [11]:
#performing data augmentation to train data
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/data/train',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 15341 images belonging to 6 classes.


In [12]:
x_test=test_datagen.flow_from_directory(directory=r'/home/wsuser/work/data/test',target_size=(64,64),batch_size=32,class_mode='categorical')

Found 6825 images belonging to 6 classes.


In [13]:
#Lets see the classes the different types of arrythmia is stored in
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

# MODEL BUILDING

In [14]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D
import tensorflow.keras

In [15]:
model=Sequential()

# Adding CNN Layers

In [16]:
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu')) #Activation Function
model.add(MaxPooling2D(pool_size=(2,2))) #Downsampling Purposes
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) #Flatten the dimensions of the image

In [17]:
model.add(Dense(32))# Deeply connected neural network layers
model.add(Dense(6,activation='softmax'))#Output layer with 6 neurons

In [18]:
#Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 32)                2

In [19]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_164/53529210.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))


Epoch 1/10
480/480 [==============================] - 90s 186ms/step - loss: 0.8845 - accuracy: 0.6983 - val_loss: 0.6509 - val_accuracy: 0.7622
Epoch 2/10
480/480 [==============================] - 87s 181ms/step - loss: 0.3313 - accuracy: 0.9006 - val_loss: 0.4420 - val_accuracy: 0.8738
Epoch 3/10
480/480 [==============================] - 85s 177ms/step - loss: 0.2590 - accuracy: 0.9238 - val_loss: 0.3705 - val_accuracy: 0.8951
Epoch 4/10
480/480 [==============================] - 86s 178ms/step - loss: 0.2152 - accuracy: 0.9398 - val_loss: 0.3378 - val_accuracy: 0.8941
Epoch 5/10
480/480 [==============================] - 86s 178ms/step - loss: 0.1823 - accuracy: 0.9473 - val_loss: 0.3742 - val_accuracy: 0.8762
Epoch 6/10
480/480 [==============================] - 88s 184ms/step - loss: 0.1550 - accuracy: 0.9525 - val_loss: 0.3282 - val_accuracy: 0.8948
Epoch 7/10
480/480 [==============================] - 85s 178ms/step - loss: 0.1478 - accuracy: 0.9557 - val_loss: 0.2508 - val_ac

# SAVING THE MODEL

In [21]:
model.save('ECG.h5')

In [22]:
!tar -zcvf arrhythmia-classification-model_new.tgz ECG.h5

ECG.h5


In [23]:
ls -1

arrhythmia-classification-model_new.tgz
data/
ECG.h5


In [24]:
#Installing amchine learning service
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 11.2 MB/s eta 0:00:01


In [41]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "url":"https://eu-gb.ml.cloud.ibm.com",
    "apikey":"G565mbpJlA5S9JVSVNwpBuQMhgBaRnVA-iOy3xiQKTAE"
}

client = APIClient(wml_credentials)

In [42]:
client=APIClient(wml_credentials)

In [43]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [44]:
space_uid=guid_from_space_name(client, 'Classificatio_of_Arrhythmia')
print("space UID="+ space_uid)

space UID=cd354d14-8e58-44b8-b1b6-5ac25a90305a


In [45]:
client.set.default_space(space_uid)

'SUCCESS'

In [46]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [47]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [48]:
model_details = client.repository.store_model(model='arrhythmia-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [49]:
model_id

'2e1b92c9-53d0-4368-b090-c4b3559e4d16'

In [51]:
client.repository.download(model_id,'my_model2.tar2.gz')

Successfully saved model content to file: 'my_model2.tar2.gz'


'/home/wsuser/work/my_model2.tar2.gz'